In [61]:
import pandas as pd

In [62]:
# 获取数据
titanic=pd.read_csv("titanic.csv")

In [63]:
titanic.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [64]:
# 筛选特征值和目标值
x=titanic[['pclass','age','sex']]
y=titanic["survived"]

In [65]:
x.head()

,pclass,age,sex
0,1st,29.0000,female
1,1st,2.0000,female
2,1st,30.0000,male
3,1st,25.0000,female
4,1st,0.9167,male


In [66]:
# 缺失值处理
x["age"].fillna(x["age"].mean(),inplace=True)

D:\Programs\Anaconda\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [67]:
# 转换成字典
x=x.to_dict(orient="records")

In [68]:
from sklearn.model_selection import train_test_split

In [69]:
# 数据集划分
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=23)

In [70]:
# 字典特征抽取
from sklearn.feature_extraction import DictVectorizer

transfer=DictVectorizer()
x_train=transfer.fit_transform(x_train)
x_test=transfer.transform(x_test)

In [71]:
from sklearn.tree import DecisionTreeClassifier,export_graphviz
# 使用决策树预估
estimator=DecisionTreeClassifier(criterion="entropy",max_depth=8)
estimator.fit(x_train,y_train)
# 模型评估
# 方法一:直接比对真实值和预测值
y_predict = estimator.predict(x_test)
print("y_predict:\n", y_predict)
print("比对结果：\n", y_test == y_predict)
# 方法二：季孙准确率
score = estimator.score(x_test, y_test)
print("准确率：\n", score)

# 可视化树
export_graphviz(estimator,out_file="titanic_tree.dot",feature_names=['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', '女性', '男性'])
    

y_predict:
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 1 1
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 1 0 0 0 1 1 0
 1 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0]
比对结果：
 25       True
576      True
84      False
1009     True
802      True
534     False
1069     True
520      True
1136    False
986      True
87       True
328      True
936      True
164      True
772      True
677      True
200      True
1226    False
1087     True
274      True
755      True
63      False
592  

## 使用随机森林进行预测

In [72]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [83]:
estimator=RandomForestClassifier()
# 添加网格搜索和交叉验证
# 准备参数
param_dict={"n_estimators":[120,200,300,500,800,1200],"max_depth": [5, 8, 15, 25, 30]}
estimator=GridSearchCV(estimator,param_grid=param_dict,cv=3)
estimator.fit(x_train,y_train)
# 5）模型评估
# 方法一:直接比对真实值和预测值
y_predict=estimator.predict(x_test)
print("y_predict:\n",y_predict)
print("比对结果：\n",y_test==y_predict)
# 方法二：计算准确率
score=estimator.score(x_test,y_test)
print("准确率：\n",score)

# 训练验证集的结果
print("最佳参数：\n",estimator.best_params_)
print("在交叉验证当中验证的最好结果：", estimator.best_score_)
print("模型K值是：", estimator.best_estimator_)
print("交叉验证的结果为：", estimator.cv_results_)

y_predict:
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 1 1
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0 1
 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 1 1 0 0 1 1 1 0
 1 0 0 0 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0]
比对结果：
 25       True
576      True
84      False
1009     True
802      True
534     False
1069     True
520      True
1136    False
986      True
87       True
328      True
936      True
164      True
772      True
677      True
200      True
1226    False
1087     True
274      True
755      True
63      False
592  